In [1]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
# Ensure TensorFlow only uses a limited amount of GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define batch sizes and number of runs
BATCH_SIZES = [1, 4, 8, 16, 32, 64]
NUM_RUNS = 5  # Runs per batch size

# Load NASNet model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))

# Create a model that outputs intermediate layer results
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow functions for performance
@tf.function
def run_inference(input_tensor):
    return model(input_tensor)

2025-02-05 11:51:30.224731: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-05 11:51:30.224753: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-05 11:51:30.224757: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-05 11:51:30.224924: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-05 11:51:30.224933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Create random input tensor
        input_tensor = tf.random.uniform((batch_size, 224, 224, 3), dtype=tf.float32)
        
        with tf.device(device):
            #layer_times = {}
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

    return results

## Experiment Logging

In [4]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []
for device in devices:
    for batch_size in BATCH_SIZES:
        try:
            tf.profiler.experimental.start(f"logs/Noise/{device[1:4]}/{batch_size}")
            final_results.extend(benchmark_model(device, batch_size))
        finally:
            tf.profiler.experimental.stop()

2025-02-05 11:51:32.446839: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:51:32.446847: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 1


2025-02-05 11:51:34.084573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-05 11:51:35.061203: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:51:35.128783: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:51:35.129939: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/1/plugins/profile/2025_02_05_11_51_35/C17586.xplane.pb
2025-02-05 11:51:35.138070: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:51:35.138076: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 4


2025-02-05 11:51:38.069338: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:51:38.138986: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:51:38.139225: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/4/plugins/profile/2025_02_05_11_51_38/C17586.xplane.pb
2025-02-05 11:51:38.147580: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:51:38.147585: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


...

2025-02-05 11:53:31.860210: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:53:31.885416: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:53:31.885674: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/16/plugins/profile/2025_02_05_11_53_31/C17586.xplane.pb
2025-02-05 11:53:31.887312: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:53:31.887315: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 32


2025-02-05 11:53:47.987022: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:53:48.011695: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:53:48.012017: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/32/plugins/profile/2025_02_05_11_53_48/C17586.xplane.pb
2025-02-05 11:53:48.014765: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 11:53:48.014772: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 64


2025-02-05 11:55:07.469432: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 11:55:07.642889: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 11:55:07.656136: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/64/plugins/profile/2025_02_05_11_55_07/C17586.xplane.pb


## Results

In [5]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.036004
1,/CPU:0,1,0.045836
2,/CPU:0,1,0.035039
3,/CPU:0,1,0.032458
4,/CPU:0,1,0.031932


In [6]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                               
                         min      mean        var       max
device batch_size                                          
/CPU:0 1            0.031932  0.036254   0.000032  0.045836
       4            0.063843  0.066628   0.000006  0.070057
       8            0.088682  0.092413   0.000012  0.095950
       16           0.146524  0.162206   0.000162  0.180534
       32           0.454610  0.485009   0.001156  0.539084
       64           1.353778  1.486213   0.018460  1.679982
/GPU:0 1            0.084557  0.090899   0.000071  0.104635
       4            0.091157  0.095121   0.000059  0.108790
       8            0.089755  0.095591   0.000086  0.112061
       16           0.089733  0.163743   0.012225  0.341366
       32           0.140988  1.634943  10.354440  7.389127
       64           0.985863  1.231255   0.086847  1.729435